<a href="https://colab.research.google.com/github/gyasifred/TW-FR-MT/blob/main/tutorials/Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Twi-French Machine Translation

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive



In this tutorial, we will illustrate how to

1. fine-tune the pre-trained [Helsinki-NLP/opus-mt-tw-fr](https://huggingface.co/Helsinki-NLP/opus-mt-tw-fr) from the [OPUS-MT](https://opus.nlpl.eu/Opus-MT/) repository for machine translation of Twi (the local Ghanaian language) to French.

2. perform translations with the fine-tuned OPUS-MT model.
3. estimate the machine translation quality with the [BLEU score](https://aclanthology.org/P02-1040.pdf), AzunreBLEU score, the [TER score](https://aclanthology.org/2006.amta-papers.25/) and the [SacreBLEU score](https://aclanthology.org/W18-6319.pdf).

*** The AzunreBLEU score is a variant of the BLEU score that was used for evaluating machine translation quality in the paper [English-Twi Parallel Corpus for Machine Translation](https://arxiv.org/pdf/2103.15625.pdf) indicating that the focus is on "adequacy" instead of "fluency" in the translations.


## [Optional] Download already-fine-tuned Twi models.


Fine-tuned Twi OPUS-MT models can be downloaded from [Google Drive](https://drive.google.com/drive/folders/13irIvPsqnryP_NJ5y6PneFKrQDJs5Qm6?usp=sharing).


This tutorial uses GPU; to use GPU, please go to EDIT on the menu bar, notebook settings, and choose GPU for the hardware accelerator, then save.

## Clone TW-FR-MT Github Repository



In [2]:
!git clone https://github.com/gyasifred/TW-FR-MT.git

Cloning into 'TW-FR-MT'...
remote: Enumerating objects: 229, done.
remote: Counting objects: 100% (229/229), done.
remote: Compressing objects: 100% (166/166), done.
remote: Total 229 (delta 114), reused 145 (delta 58), pack-reused 0
Receiving objects: 100% (229/229), 1.10 MiB | 1.00 MiB/s, done.
Resolving deltas: 100% (114/114), done.


In [3]:
%cd TW-FR-MT/

/content/TW-FR-MT


In [4]:
!ls

__init__.py  MT_systems  requirements.txt  TW_FR_EN_corpus
LICENSE      README.md	 tutorials


## Install Dependencies


In [5]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 80.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 95.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 74.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Training, Validation, and Test sets




Get the help message for how to use any script by passing the -h flag.

In [ ]:
#!python /content/TW-FR-MT/TW_FR_EN_corpus/scripts/train_test_split.py -h

In [23]:
!python /content/TW-FR-MT/TW_FR_EN_corpus/scripts/train_test_split.py \
/content/TW-FR-MT/TW_FR_EN_corpus/data/total/total_tw.txt \
/content/TW-FR-MT/TW_FR_EN_corpus/data/total/total_en.txt \
/content/TW-FR-MT/TW_FR_EN_corpus/data/total/total_fr.txt \
--id_tw tw \
--id_en en \
--id_fr fr \
--train_ouput_path /content/TW-FR-MT/TW_FR_EN_corpus/data/training \
--test_ouput_path /content/TW-FR-MT/TW_FR_EN_corpus/data/test \
--val_ouput_path  /content/TW-FR-MT/TW_FR_EN_corpus/data/validation \
--val_set True

SUCCESS


## Fine-tuning the OPUS-MT Model


In [ ]:
#!python /content/TW-FR-MT/MT_systems/opus/fine_tune_opus.py -h

In [ ]:
!python /content/TW-FR-MT/MT_systems/opus/fine_tune_opus.py \
Helsinki-NLP/opus-mt-tw-fr \
/content/TW-FR-MT/TW_FR_EN_corpus/data/training/train_tw.txt \
/content/TW-FR-MT/TW_FR_EN_corpus/data/training/train_fr.txt \
/content/TW-FR-MT/TW_FR_EN_corpus/data/validation/val_tw.txt \
/content/TW-FR-MT/TW_FR_EN_corpus/data/validation/val_fr.txt \
tw \
fr \
OPUS-mt-tw-fr-tuned-1 \
--max_length 128 \
--batch_size 8 \
--epoch 24 \
--warmup_steps 10 \
--savedir /content/drive/MyDrive/MT

2023-06-10 16:14:51.612006: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
MODEL PATH: /content/drive/MyDrive/MT
FINED-TURNED MODEL NAME: OPUS-mt-tw-fr-tuned-1
  4% 1071/25704 [01:18<26:42, 15.37it/s]
  0% 0/134 [00:00<?, ?it/s]
  1% 1/134 [00:00<00:40,  3.31it/s]
  1% 2/134 [00:00<00:38,  3.39it/s]
  2% 3/134 [00:00<00:34,  3.81it/s]
  3% 4/134 [00:01<00:35,  3.61it/s]
  4% 5/134 [00:01<00:33,  3.84it/s]
  4% 6/134 [00:01<00:32,  3.95it/s]
  5% 7/134 [00:01<00:36,  3.44it/s]
  6% 8/134 [00:02<00:35,  3.56it/s]
  7% 9/134 [00:02<00:43,  2.86it/s]
  7% 10/134 [00:03<00:42,  2.90it/s]
  8% 11/134 [00:03<00:45,  2.71it/s]
  9% 12/134 [00:03<00:42,  2.85it/s]
 10% 13/134 [00:04<00:41,  2.95it/s]
 10% 14/134 [00:04<00:42,  2.85it/s]
 11% 15/134 [00:04<00:43,  2.71it/s]
 12% 16/134 [00:05<00:47,  2.47it/s]
 13% 17/134 [00:05<00:49,  2.38it/s]
 13% 18/134 [00:06<00:53,  2.18it/s]
 14% 19/134 [00:06<00:49,  2.34it/s]
 15% 20/134 [00:08<01:23,  1


## Translate with the Fine-tuned OPUS-MT Model



In [7]:
!python /content/TW-FR-MT/MT_systems/opus/opus_direct_translate.py \
/content/drive/MyDrive/MT/OPUS-mt-tw-fr-tuned-1 \
/content/TW-FR-MT/TW_FR_EN_corpus/data/test/test_tw.txt \
--output_name tw-fr-opus-translate

2023-06-29 21:21:49.505838: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Evaluate the Translation Quality of the OPUS-MT Model

### BLEU Score

In [8]:
!python /content/TW-FR-MT/MT_systems/evalution_scripts/get_bleu.py \
tw-fr-opus-translate \
/content/TW-FR-MT/TW_FR_EN_corpus/data/test/test_fr.txt

2023-06-29 21:35:57.146419: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
('2-GRAMS: 0.55', '3-GRAMS: 0.44', '4-GRAMS: 0.36')


### AzunreBLEU Score

In [9]:
!python /content/TW-FR-MT/MT_systems/evalution_scripts/get_bleu.py \
tw-fr-opus-translate \
/content/TW-FR-MT/TW_FR_EN_corpus/data/test/test_fr.txt \
--azunre True

2023-06-29 21:36:09.182527: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
BLEU SCORE: 0.76


### SacreBLEU Score

In [10]:
!python /content/TW-FR-MT/MT_systems/evalution_scripts/get_sacrebleu.py \
tw-fr-opus-translate \
/content/TW-FR-MT/TW_FR_EN_corpus/data/test/test_fr.txt

2023-06-29 21:36:19.534474: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
SacreBLEU SCORE: 36.55


TER Score

In [11]:
!python /content/TW-FR-MT/MT_systems/evalution_scripts/ter.py \
tw-fr-opus-translate \
/content/TW-FR-MT/TW_FR_EN_corpus/data/test/test_fr.txt

2023-06-29 21:36:27.926837: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
TER SCORE: 61.05


## Pre-trained OPUST-MT model

In [12]:
!python /content/TW-FR-MT/MT_systems/opus/opus_direct_translate.py \
Helsinki-NLP/opus-mt-tw-fr \
/content/TW-FR-MT/TW_FR_EN_corpus/data/test/test_tw.txt \
--output_name tw-fr-opus-translate-pretrained

2023-06-29 21:36:37.207113: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [24]:
!python /content/TW-FR-MT/MT_systems/evalution_scripts/get_bleu.py \
tw-fr-opus-translate-pretrained \
/content/TW-FR-MT/TW_FR_EN_corpus/data/test/test_fr.txt

2023-06-29 22:27:15.712607: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
('2-GRAMS: 0.39', '3-GRAMS: 0.29', '4-GRAMS: 0.22')


In [25]:
!python /content/TW-FR-MT/MT_systems/evalution_scripts/get_bleu.py \
tw-fr-opus-translate-pretrained \
/content/TW-FR-MT/TW_FR_EN_corpus/data/test/test_fr.txt \
--azunre True

2023-06-29 22:27:30.694960: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
BLEU SCORE: 0.62


In [26]:
!python /content/TW-FR-MT/MT_systems/evalution_scripts/get_sacrebleu.py \
tw-fr-opus-translate-pretrained \
/content/TW-FR-MT/TW_FR_EN_corpus/data/test/test_fr.txt

2023-06-29 22:27:39.836218: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
SacreBLEU SCORE: 20.91


TER score

In [31]:
!python /content/TW-FR-MT/MT_systems/evalution_scripts/ter.py \
tw-fr-opus-translate-pretrained \
/content/TW-FR-MT/TW_FR_EN_corpus/data/test/test_fr.txt

2023-06-29 22:44:23.100810: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
TER SCORE: 91.51


## Google Translate API



### Translate with the Google Translate API


In [16]:
! python /content/TW-FR-MT/MT_systems/Google_MT/googleAPIdirect_translate.py \
/content/TW-FR-MT/TW_FR_EN_corpus/data/test/test_tw.txt \
ak\
fr\
--output_name twi-fr-google-translate


### Evaluate the Translation Quality of the Google API Translation


#### BLEU Score

In [27]:
!python /content/TW-FR-MT/MT_systems/evalution_scripts/get_bleu.py \
twi-fr-google-translate \
/content/TW-FR-MT/TW_FR_EN_corpus/data/test/test_fr.txt

2023-06-29 22:27:49.207745: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
('2-GRAMS: 0.60', '3-GRAMS: 0.49', '4-GRAMS: 0.41')


#### AzunreBLEU Score

In [28]:
!python /content/TW-FR-MT/MT_systems/evalution_scripts/get_bleu.py \
twi-fr-google-translate \
/content/TW-FR-MT/TW_FR_EN_corpus/data/test/test_fr.txt \
--azunre True

2023-06-29 22:27:57.845214: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
BLEU SCORE: 0.79


#### SacreBLEU Score

In [29]:
!python /content/TW-FR-MT/MT_systems/evalution_scripts/get_sacrebleu.py \
twi-fr-google-translate \
/content/TW-FR-MT/TW_FR_EN_corpus/data/test/test_fr.txt

2023-06-29 22:28:07.100360: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
SacreBLEU SCORE: 44.40


TER Score

In [30]:
!python /content/TW-FR-MT/MT_systems/evalution_scripts/ter.py \
twi-fr-google-translate \
/content/TW-FR-MT/TW_FR_EN_corpus/data/test/test_fr.txt

2023-06-29 22:28:16.835978: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
TER SCORE: 55.59


## OPUS Pivot Translation


To perform pivot translation, you will need to fine-tune additional models.<br>
You can alternatively download fine-tuned models from our project from Google [Drive](https://drive.google.com/drive/folders/13irIvPsqnryP_NJ5y6PneFKrQDJs5Qm6?usp=sharing).


$<source>$ -> $<English>$ ->$<Target>$


We will demonstrate pivot Twi -> English -> French translation.<br>
In this tutorial, we use our fine-tuned models. To use pre-trained OPUS-MT models, pass the name of the OPUS-MT model as shown below.<br>

For French -> English -> Twi using Pre-trained OPUS-MT models
```
!python /content/TW-FR-MT/MT_systems/opus/opus_pivot_translate.py \
Helsinki-NLP/opus-mt-fr-en \
Helsinki-NLP/opus-mt-en-tw \
/content/TW-FR-MT/TW_FR_EN_corpus/data/test/test_fr.txt \
--to_console True

```




In [21]:
!python /content/TW-FR-MT/MT_systems/opus/opus_pivot_translate.py \
/content/drive/MyDrive/MT/OPUS-mt-tw-en-tuned \
/content/drive/MyDrive/MT/OPUS-mt-en-fr-tuned \
/content/TW-FR-MT/TW_FR_EN_corpus/data/test/test_tw.txt \
--to_console True

2023-06-29 21:55:24.992812: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Source: Wayɛ krado.
Pivot: He is ready.
Target: Il est prêt.

Source: Ɔnsua ade pii.
Pivot: He wouldn't study enough.
Target: Il n'étudierait pas assez.

Source: Smith buae sɛ wanu ne ho.
Pivot: Asamoah answered he was sorry.
Target: Asamoah a répondu qu'il était désolé.

Source: Fa safoa no ma wo nua.
Pivot: Give the key to your brother.
Target: Donne la clé à ton frère.

Source: Ɛyɛ asɛm a ɛhaw yɛn nyinaa.
Pivot: That's all what's worried.
Target: C'est tout ce qui est inquiet.

Source: Wamma m'asɛmmisa no ho mmuae.
Pivot: He didn't answer my question.
Target: Il n'a pas répondu ma question.

Source: Seesei minni sika.
Pivot: I have no money now.
Target: Je n'ai plus d'argent maintenant.

Source: Ná ɛyɛ awerɛhosɛm.
Pivot: It was sad nervous.
Target: C'était triste et nerveux.

Source: Me nuabarima no wɔ sika pii a obetumi de atɔ kar.
Pivot: My brother has more m